##### Copyright 2019 The TensorFlow Authors.

In [1]:
import tensorflow as tf; 
print(tf.__version__)

2.10.1


In [2]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](images/beginner/run_cell_icon.png)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.1


In [5]:

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.14116366,  0.45004767, -0.23359188, -0.7618405 , -0.39902514,
         0.07490199, -0.43750775, -0.1146087 , -0.35857275,  0.3136936 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.12340812, 0.16807005, 0.08483782, 0.05002348, 0.07190228,
        0.11549591, 0.06918786, 0.09555715, 0.07487053, 0.14664678]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.1585202

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=55)

Epoch 1/55
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2908 - accuracy: 0.9161
Epoch 2/55
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1435 - accuracy: 0.9568
Epoch 3/55
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1077 - accuracy: 0.9669
Epoch 4/55
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0890 - accuracy: 0.9725
Epoch 5/55
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0763 - accuracy: 0.9760
Epoch 6/55
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0658 - accuracy: 0.9793
Epoch 7/55
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0578 - accuracy: 0.9813
Epoch 8/55
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0534 - accuracy: 0.9828
Epoch 9/55
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0477 - accuracy: 0.9843
Epoch 10/55
1875/1875 [==============================] - 5s 3ms/step - loss: 0.044

The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [16]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.1288 - accuracy: 0.9797 - 590ms/epoch - 2ms/step


[0.1288061887025833, 0.9797000288963318]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [28]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [29]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.79283305e-07, 1.32197409e-08, 1.55715315e-05, 2.02358438e-04,
        7.58437410e-11, 4.16019077e-08, 5.11944480e-12, 9.99774277e-01,
        1.49787795e-07, 7.48567345e-06],
       [5.21329957e-09, 1.44614751e-04, 9.99846578e-01, 7.23749145e-06,
        2.72502046e-13, 1.04992715e-07, 4.91884506e-08, 1.16196983e-12,
        1.38855842e-06, 1.29977111e-13],
       [1.95801860e-07, 9.99085426e-01, 1.44428617e-04, 5.66570634e-06,
        2.39041619e-05, 1.86133047e-06, 9.82684105e-06, 6.26007270e-04,
        1.02378603e-04, 2.79163487e-07],
       [9.99756634e-01, 1.75441195e-09, 1.95629073e-05, 4.71448948e-06,
        8.59671491e-06, 1.77700840e-05, 1.76105634e-04, 1.44872347e-05,
        4.28303309e-07, 1.71425916e-06],
       [2.14805709e-06, 4.04512939e-08, 8.55208145e-06, 5.08110134e-07,
        9.98082519e-01, 2.62736330e-06, 2.42776077e-05, 2.99232721e-04,
        1.20684888e-07, 1.57989096e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
